In [2]:
import numpy as np
from numpy import genfromtxt
import glob
from keras.preprocessing.sequence import pad_sequences
from sklearn.cross_validation import train_test_split
import keras
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import sys
pd.set_option('display.max_rows', 1000)
import random
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from scipy import stats

In [3]:
data_folder = "data\\RawData\\"
acc_path = data_folder + "acc_*.txt"
gyro_path = data_folder + "gyro_*.txt"
label_path = data_folder + "labels.txt"

df_labels = pd.read_csv(label_path, delimiter=r"\s+")

df_main = pd.DataFrame(columns=['acc0','acc1','acc2','gyro0','gyro1','gyro2','exp_id','label_id'])

acc_files = glob.glob(acc_path)
for acc_file in acc_files:
    
    exp_id = acc_file[20:22]
    df_acc = pd.read_csv(acc_file, header=None, delimiter=r"\s+", prefix="acc")
    gyro_file = data_folder + "gyro" + acc_file[len(data_folder)+3:]
    df_gyro = pd.read_csv(gyro_file, header=None, delimiter=r"\s+", prefix="gyro")
    
    if len(df_acc) != len(df_gyro):
        print ("Error: lenght of acc and gyro files are different for exp " + exp_id + ": " 
               + str(len(df_acc)) + " " + str(len(df_gyro)))
    
    df_concat = pd.concat([df_acc, df_gyro], axis = 1)
    df_concat["exp_id"] = int(exp_id)
    df_concat["label_id"] = 0

    df_exp_labels = df_labels.loc[df_labels['exp_id'] == int(exp_id)]

    for index, row in df_exp_labels.iterrows():
        start_point = row['start_point']
        end_point   = row['end_point']
        label_id    = row['act_id']
        df_concat.loc[start_point:end_point , 'label_id'] = label_id

    df_main = df_main.append(df_concat, ignore_index = True)

In [4]:
split_exp_id = 47
df_continuous = df_main.loc[df_main['exp_id'] >= split_exp_id]
df_continuous = df_continuous.reset_index(drop=True)

df_OVA = df_main.loc[df_main['exp_id'] < split_exp_id]
df_OVA = df_OVA.reset_index(drop=True)
df_OVA

,acc0,acc1,acc2,gyro0,gyro1,gyro2,exp_id,label_id
0,0.918056,-0.112500,0.509722,-0.054978,-0.069639,-0.030849,1,0
1,0.911111,-0.093056,0.537500,-0.012523,0.019242,-0.038485,1,0
2,0.881944,-0.086111,0.513889,-0.023518,0.276417,0.006414,1,0
3,0.881944,-0.086111,0.513889,-0.093462,0.367741,0.001222,1,0
4,0.879167,-0.100000,0.505556,-0.124311,0.476780,-0.022907,1,0
5,0.888889,-0.105556,0.512500,-0.100487,0.519846,-0.067501,1,0
6,0.862500,-0.101389,0.509722,-0.149357,0.481056,-0.092546,1,0
7,0.861111,-0.104167,0.501389,-0.211054,0.389121,-0.074831,1,0
8,0.854167,-0.108333,0.527778,-0.222355,0.267864,-0.051924,1,0
9,0.851389,-0.101389,0.552778,-0.173791,0.207083,-0.032070,1,0


In [5]:
window_size = 128
window_move_size = window_size // 2

In [6]:
segmeneted_activities = [ 'N/A',[],[],[],[],[],[] ]

for i in range(1 , split_exp_id):
    
    df_exp = df_OVA.loc[df_OVA['exp_id'] == i]
    df_exp = df_exp.reset_index(drop=True)
    
    index = 0
    continu = True
    while (continu):
        
        if index >= len(df_exp):
            break
        
        df_window = df_exp.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break
        
        window_label_ids = set(df_window['label_id'].values)
        if (len(window_label_ids) > 1) or \
           (len(window_label_ids)==1 and next(iter(window_label_ids))==0) or \
           (len(window_label_ids)==1 and next(iter(window_label_ids))>6):
            index = index + window_move_size
            continue
    
        na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
        window_label_id = next(iter(window_label_ids))
        
        segmeneted_activities[window_label_id].append(na_window)
        index = index + window_move_size

In [7]:
for i in range(1,7):
    segmeneted_activities[i] = np.array(segmeneted_activities[i])

In [8]:
for i in range(1,7):
    print ("Act_id:" , i , " " , segmeneted_activities[i].shape)

Act_id: 1   (1227, 128, 6)
Act_id: 2   (1069, 128, 6)
Act_id: 3   (945, 128, 6)
Act_id: 4   (1236, 128, 6)
Act_id: 5   (1386, 128, 6)
Act_id: 6   (1363, 128, 6)


In [9]:
def get_binary_X_and_Y_subsampled(act_id):
    
    X , Y = get_binary_X_and_Y (act_id)
    
    total_datapoints = X.shape[0]
    plus_datapoints = segmeneted_activities[act_id].shape[0]
    alpha = (total_datapoints - plus_datapoints) / plus_datapoints
    New_X = []
    New_Y = []
   
    for i in range(total_datapoints):
        
        u = np.random.uniform(0,1)
        if (Y[i]==1) or (u < (1/alpha)):
            New_X.append(X[i])
            New_Y.append(Y[i])

    New_X = np.array(New_X)
    New_Y = np.array(New_Y)
    
    return New_X , New_Y

In [10]:
def get_binary_X_and_Y (act_id):

    X = np.concatenate((segmeneted_activities[1], segmeneted_activities[2], segmeneted_activities[3]))
    X = np.concatenate((           X            , segmeneted_activities[4], segmeneted_activities[5]))
    X = np.concatenate((           X            , segmeneted_activities[6]))

    ys = [ 'N/A',None,None,None,None,None,None ]
    for i in range(1,7):
        size = segmeneted_activities[i].shape[0]
        if act_id == i:
            y = np.full (size , 1)
        else:
            y = np.full (size , 0)
        ys[i] = y

    Y = np.concatenate((ys[1], ys[2], ys[3]))
    Y = np.concatenate((  Y  , ys[4], ys[5]))
    Y = np.concatenate((  Y  , ys[6]))
    
    return (X , Y)

In [42]:
def create_model (data_input_shape):

    LSTM_output_dim = 200
    
    model = Sequential()
    
    #model.add(LSTM(LSTM_output_dim, return_sequences=True, name='LSTM1', input_shape=data_input_shape))    
    #model.add(LSTM(LSTM_output_dim, return_sequences=True, name='LSTM2'))
    #model.add(LSTM(LSTM_output_dim, return_sequences=False, name='LSTM3'))
    
    model.add(LSTM(LSTM_output_dim, return_sequences=False, name='LSTM1', input_shape=data_input_shape))
    
    model.add(Dense(15, activation='sigmoid',name='Hidden_Layer'))
    
    #model.add(Dense(3, activation='softmax', name='Softmax_layer'))
    model.add(Dense(1, activation='sigmoid',name='Dense_out'))
    
    #model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

    return model

In [27]:
def train_binary_classifier(act_id):
    #(X , Y) = get_binary_X_and_Y (act_id)
    (X , Y) = get_binary_X_and_Y_subsampled (act_id)
    
    X , Y = shuffle(X , Y)

    X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25)
    X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=0.333)

    data_input_shape = X.shape[1:3]
    model = create_model(data_input_shape)
    print (model.summary()  , "\n")

    patience = 10
    callbacks=[EarlyStopping(monitor='val_loss',patience=patience,verbose=1)]
    history = model.fit(X_train,y_train,
                        validation_data=[X_valid,y_valid],
                        epochs=100, 
                        batch_size=100
                        #,callbacks=callbacks
                       )

    score = model.evaluate(X_test, y_test, verbose=0)
    print('\nTest loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model

In [28]:
#training binary classifier for activity 1 (WALKING)
act_id = 1
model_1 = train_binary_classifier(act_id)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 100)               42800     
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                1515      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 44,331
Trainable params: 44,331
Non-trainable params: 0
_________________________________________________________________
None 

Train on 1232 samples, validate on 616 samples
Epoch 1/100
1232/1232 [==============================] - 4s 3ms/step - loss: 0.6752 - acc: 0.6161 - val_loss: 0.6420 - val_acc: 0.6623
Epoch 2/100
1232/1232 [==============================] - 2s 2ms/step - loss: 0.6190 - acc: 0.6648 - val_loss: 0.6100 - val_acc: 0.6899
Epoch 3/100
1232/1232 [==============================] - 2s 2ms/step - loss: 0

1232/1232 [==============================] - 2s 2ms/step - loss: 0.3289 - acc: 0.8774 - val_loss: 0.4365 - val_acc: 0.8312
Epoch 56/100
1232/1232 [==============================] - 2s 2ms/step - loss: 0.3309 - acc: 0.8758 - val_loss: 0.3839 - val_acc: 0.8442
Epoch 57/100
1232/1232 [==============================] - 2s 2ms/step - loss: 0.2969 - acc: 0.8920 - val_loss: 0.3630 - val_acc: 0.8604
Epoch 58/100
1232/1232 [==============================] - 2s 2ms/step - loss: 0.2933 - acc: 0.8920 - val_loss: 0.3243 - val_acc: 0.8734
Epoch 59/100
1232/1232 [==============================] - 2s 2ms/step - loss: 0.2836 - acc: 0.8977 - val_loss: 0.3069 - val_acc: 0.8896
Epoch 60/100
1232/1232 [==============================] - 2s 2ms/step - loss: 0.2757 - acc: 0.8985 - val_loss: 0.2997 - val_acc: 0.8880
Epoch 61/100
1232/1232 [==============================] - 2s 2ms/step - loss: 0.2747 - acc: 0.8985 - val_loss: 0.2955 - val_acc: 0.8864
Epoch 62/100
1232/1232 [==============================] - 2s 

In [38]:
#training binary classifier for activity 2 (WALKING_UPSTAIRS)
act_id = 2
model_2 = train_binary_classifier(act_id)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 100)               42800     
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                1515      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 44,331
Trainable params: 44,331
Non-trainable params: 0
_________________________________________________________________
None 

Train on 1069 samples, validate on 534 samples
Epoch 1/100
1069/1069 [==============================] - 5s 4ms/step - loss: 0.6720 - acc: 0.5650 - val_loss: 0.6205 - val_acc: 0.7397
Epoch 2/100
1069/1069 [==============================] - 3s 2ms/step - loss: 0.5149 - acc: 0.8316 - val_loss: 0.4840 - val_acc: 0.8127
Epoch 3/100
1069/1069 [==============================] - 3s 2ms/step - loss: 0

1069/1069 [==============================] - 3s 2ms/step - loss: 0.3040 - acc: 0.8999 - val_loss: 0.3070 - val_acc: 0.9026
Epoch 56/100
1069/1069 [==============================] - 3s 3ms/step - loss: 0.3087 - acc: 0.9046 - val_loss: 0.3341 - val_acc: 0.9007
Epoch 57/100
1069/1069 [==============================] - 3s 3ms/step - loss: 0.3118 - acc: 0.9036 - val_loss: 0.3186 - val_acc: 0.8989
Epoch 58/100
1069/1069 [==============================] - 3s 3ms/step - loss: 0.3036 - acc: 0.9046 - val_loss: 0.3006 - val_acc: 0.9045
Epoch 59/100
1069/1069 [==============================] - 3s 3ms/step - loss: 0.2783 - acc: 0.9102 - val_loss: 0.2894 - val_acc: 0.9120
Epoch 60/100
1069/1069 [==============================] - 3s 3ms/step - loss: 0.2596 - acc: 0.9177 - val_loss: 0.3068 - val_acc: 0.9026
Epoch 61/100
1069/1069 [==============================] - 3s 3ms/step - loss: 0.2659 - acc: 0.9158 - val_loss: 0.2791 - val_acc: 0.9176
Epoch 62/100
1069/1069 [==============================] - 3s 

In [39]:
#training binary classifier for activity 3 (WALKING_DOWNSTAIRS)
act_id = 3
model_3 = train_binary_classifier(act_id)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 100)               42800     
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                1515      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 44,331
Trainable params: 44,331
Non-trainable params: 0
_________________________________________________________________
None 

Train on 959 samples, validate on 479 samples
Epoch 1/100
959/959 [==============================] - 4s 4ms/step - loss: 0.7300 - acc: 0.4995 - val_loss: 0.6970 - val_acc: 0.4906
Epoch 2/100
959/959 [==============================] - 2s 3ms/step - loss: 0.6154 - acc: 0.6684 - val_loss: 0.5196 - val_acc: 0.7766
Epoch 3/100
959/959 [==============================] - 2s 2ms/step - loss: 0.5321 -

959/959 [==============================] - 3s 3ms/step - loss: 0.3440 - acc: 0.8551 - val_loss: 0.3652 - val_acc: 0.8392
Epoch 57/100
959/959 [==============================] - 3s 3ms/step - loss: 0.3224 - acc: 0.8686 - val_loss: 0.3499 - val_acc: 0.8476
Epoch 58/100
959/959 [==============================] - 3s 3ms/step - loss: 0.3184 - acc: 0.8655 - val_loss: 0.3361 - val_acc: 0.8476
Epoch 59/100
959/959 [==============================] - 3s 3ms/step - loss: 0.3455 - acc: 0.8446 - val_loss: 0.4103 - val_acc: 0.8476
Epoch 60/100
959/959 [==============================] - 2s 3ms/step - loss: 0.4780 - acc: 0.8071 - val_loss: 0.4015 - val_acc: 0.8351
Epoch 61/100
959/959 [==============================] - 3s 3ms/step - loss: 0.3538 - acc: 0.8457 - val_loss: 0.3894 - val_acc: 0.8225
Epoch 62/100
959/959 [==============================] - 3s 3ms/step - loss: 0.3557 - acc: 0.8603 - val_loss: 0.3734 - val_acc: 0.8330
Epoch 63/100
959/959 [==============================] - 3s 3ms/step - loss:

In [40]:
#training binary classifier for activity 4 (SITTING)
act_id = 4
model_4 = train_binary_classifier(act_id)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 100)               42800     
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                1515      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 44,331
Trainable params: 44,331
Non-trainable params: 0
_________________________________________________________________
None 

Train on 1232 samples, validate on 616 samples
Epoch 1/100
1232/1232 [==============================] - 5s 4ms/step - loss: 0.6806 - acc: 0.5779 - val_loss: 0.6636 - val_acc: 0.5308
Epoch 2/100
1232/1232 [==============================] - 3s 3ms/step - loss: 0.6284 - acc: 0.6307 - val_loss: 0.5745 - val_acc: 0.7532
Epoch 3/100
1232/1232 [==============================] - 3s 3ms/step - loss: 0

1232/1232 [==============================] - 4s 3ms/step - loss: 0.2302 - acc: 0.9318 - val_loss: 0.2516 - val_acc: 0.9221
Epoch 56/100
1232/1232 [==============================] - 4s 3ms/step - loss: 0.2574 - acc: 0.9164 - val_loss: 0.2636 - val_acc: 0.9010
Epoch 57/100
1232/1232 [==============================] - 4s 3ms/step - loss: 0.2649 - acc: 0.9237 - val_loss: 0.2833 - val_acc: 0.9205
Epoch 58/100
1232/1232 [==============================] - 4s 3ms/step - loss: 0.3530 - acc: 0.9042 - val_loss: 0.2594 - val_acc: 0.9091
Epoch 59/100
1232/1232 [==============================] - 4s 3ms/step - loss: 0.2433 - acc: 0.9213 - val_loss: 0.2472 - val_acc: 0.9188
Epoch 60/100
1232/1232 [==============================] - 4s 3ms/step - loss: 0.2472 - acc: 0.9156 - val_loss: 0.2862 - val_acc: 0.8945
Epoch 61/100
1232/1232 [==============================] - 4s 3ms/step - loss: 0.2374 - acc: 0.9213 - val_loss: 0.2309 - val_acc: 0.9286
Epoch 62/100
1232/1232 [==============================] - 5s 

In [43]:
#training binary classifier for activity 5 (STANDING)
act_id = 5
model_5 = train_binary_classifier(act_id)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 200)               165600    
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                3015      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 168,631
Trainable params: 168,631
Non-trainable params: 0
_________________________________________________________________
None 

Train on 1363 samples, validate on 681 samples
Epoch 1/100
1363/1363 [==============================] - 12s 9ms/step - loss: 0.6512 - acc: 0.6354 - val_loss: 0.6741 - val_acc: 0.6725
Epoch 2/100
1363/1363 [==============================] - 11s 8ms/step - loss: 0.5672 - acc: 0.7131 - val_loss: 0.5785 - val_acc: 0.6725
Epoch 3/100
1363/1363 [==============================] - 11s 8ms/step - lo

1363/1363 [==============================] - 13s 10ms/step - loss: 0.7543 - acc: 0.5356 - val_loss: 0.6974 - val_acc: 0.4273
Epoch 56/100
1363/1363 [==============================] - 12s 8ms/step - loss: 0.6768 - acc: 0.5576 - val_loss: 0.6767 - val_acc: 0.5977
Epoch 57/100
1363/1363 [==============================] - 11s 8ms/step - loss: 0.6714 - acc: 0.5583 - val_loss: 0.6723 - val_acc: 0.6079
Epoch 58/100
1363/1363 [==============================] - 10s 8ms/step - loss: 0.6550 - acc: 0.6434 - val_loss: 0.6567 - val_acc: 0.4479
Epoch 59/100
1363/1363 [==============================] - 10s 8ms/step - loss: 0.6567 - acc: 0.6038 - val_loss: 0.6488 - val_acc: 0.7048
Epoch 60/100
1363/1363 [==============================] - 11s 8ms/step - loss: 0.6588 - acc: 0.5891 - val_loss: 0.8651 - val_acc: 0.3054
Epoch 61/100
1363/1363 [==============================] - 11s 8ms/step - loss: 0.7046 - acc: 0.5583 - val_loss: 0.5309 - val_acc: 0.7812
Epoch 62/100
1363/1363 [=============================

In [18]:
#training binary classifier for activity 6 (LAYING)
act_id = 6
model_6 = train_binary_classifier(act_id)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 100)               42800     
_________________________________________________________________
Hidden_Layer (Dense)         (None, 15)                1515      
_________________________________________________________________
Dense_out (Dense)            (None, 1)                 16        
Total params: 44,331
Trainable params: 44,331
Non-trainable params: 0
_________________________________________________________________
None 

Train on 1378 samples, validate on 689 samples
Epoch 1/50
1378/1378 [==============================] - 3s 3ms/step - loss: 0.7870 - acc: 0.5015 - val_loss: 0.5141 - val_acc: 0.4964
Epoch 2/50
1378/1378 [==============================] - 3s 2ms/step - loss: 0.3278 - acc: 0.9311 - val_loss: 0.2564 - val_acc: 0.9594
Epoch 3/50
1378/1378 [==============================] - 3s 2ms/step - loss: 0.22

In [89]:
model_1.save('model_1.h5')
model_2.save('model_2.h5')
model_3.save('model_3.h5')
model_4.save('model_4.h5')
model_5.save('model_5.h5')
model_6.save('model_6.h5')

In [30]:
def predict(X_predict):

    X_predict = np.reshape(X_predict , (1,X_predict.shape[0],X_predict.shape[1]))

    scores = [0, 0, 0, 0, 0, 0]
    for i in range(6):
        pred_probs = models[i].predict_proba(X_predict)
        scores[i] = pred_probs[0][0]

    threshold = 0.5
    if max(scores) < threshold:
        label_id = 0
    else:
        label_id = scores.index(max(scores)) + 1
    
    return label_id

In [44]:
# =================
#df_continuous2 = df_continuous.loc[df_continuous['exp_id'] == 47]
df_continuous2 = df_continuous
# =================

models = []
models.append(model_1)
models.append(model_2)
models.append(model_3)
models.append(model_4)
models.append(model_5)
models.append(model_6)

index = 0
continu = True
correct_preds = 0
total_preds = 0
while (continu):
        
    if index >= len(df_continuous2):
        break
        
    df_window = df_continuous2.loc[index:index+window_size-1]
    if len(df_window) < window_size:
        break
        
    mode_win_labels = stats.mode(df_window['label_id'].values)
    window_label_id = mode_win_labels[0][0]
    
    if window_label_id == 0:
        print ("Window Label: 0")
        index = index + window_move_size
        continue
        
    na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
    pred_label_id = predict(na_window)

    print ("Window Label: " + str(window_label_id) + "  Predicted Label: " + str(pred_label_id))
        
    total_preds += 1
        
    if (pred_label_id == window_label_id) or \
       (window_label_id>6 and pred_label_id==0) or \
       (window_label_id==7 and (pred_label_id==5 or pred_label_id==4)) or \
       (window_label_id==8 and (pred_label_id==4 or pred_label_id==5)) or \
       (window_label_id==9 and (pred_label_id==4 or pred_label_id==6)) or \
       (window_label_id==10 and (pred_label_id==6 or pred_label_id==4)) or \
       (window_label_id==11 and (pred_label_id==5 or pred_label_id==6)) or \
       (window_label_id==12 and (pred_label_id==6 or pred_label_id==5)):
        
        correct_preds += 1
        
    index = index + window_move_size
            
print ("\nContinuous Accuracy: " + str(correct_preds/total_preds))

Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 7  Predicted Label: 5
Window Label: 7  Predicted Label: 3
Window Label: 7  Predicted Label: 0
Window Label: 7  Predicted Label: 1
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Wind

Window Label: 3  Predicted Label: 3
Window Label: 0
Window Label: 0
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 3
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 0
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label

Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 0
Window Label: 2  Predicted Label: 0
Window Label: 2  Predicted Label: 5
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 3  Predicted Label: 0
Window Label: 3  Predicted Label: 0
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 5
Window Label: 3  Predicted Label: 0
Window Label: 3  Predicted Label: 3
Window Label: 0
Window Label: 0
Window Label: 2  Predicted Label: 5
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 0
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Pre

Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 3
Window Label: 1  Predicted Label: 3
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 2
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 5
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Pre

Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 12  Predicted Label: 6
Window Label: 12  Predicted Label: 6
Window Label: 12  Predicted Label: 6
Window Label: 12  Predicted Label: 3
Window Label: 12  Predicted Label: 1
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 0
Window Label: 1  Predicted Label: 3
Window Label: 1  Predicted Label: 5
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 3
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Win

...

Window Label: 11  Predicted Label: 0
Window Label: 11  Predicted Label: 4
Window Label: 11  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 10  Predicted Label: 6
Window Label: 10  Predicted Label: 6
Window Label: 10  Predicted Label: 6
Window Label: 4  Predicted Label: 5
Window Label: 4  Predi

Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 11  Predicted Label: 2
Window Label: 11  Predicted Label: 0
Window Label: 11  Predicted Label: 0
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicte

Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 0
Window Label: 7  Predicted Label: 5
Window Label: 7  Predicted Label: 0
Window Label: 4  Predicted Label: 5
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 3
Window Label: 8  Predicted Label: 3
Window Label: 8  Predicted Label: 0
Window Label: 5  Predicted L

Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 5
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 3
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 2  Predicted Label: 1
Window Label: 2  Predicted Label: 1
Window Label: 2  Predicted Label: 1
Window Label: 2  Predicted Label: 1
Window Label: 2  Predicted Label: 1
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predict

Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 1
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 5
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Wind

In [ ]:
# =================
#df_continuous2 = df_continuous.loc[df_continuous['exp_id'] == 47]
df_continuous2 = df_continuous
# =================

models = []
models.append(model_1)
models.append(model_2)
models.append(model_3)
models.append(model_4)
models.append(model_5)
models.append(model_6)

index = 0
continu = True
correct_preds = 0
total_preds = 0
while (continu):
        
    if index >= len(df_continuous2):
        break
        
    df_window = df_continuous2.loc[index:index+window_size-1]
    if len(df_window) < window_size:
        break
        
    mode_win_labels = stats.mode(df_window['label_id'].values)
    window_label_id = mode_win_labels[0][0]
    
    if window_label_id == 0:
        print ("Window Label: 0")
        index = index + window_move_size
        continue
        
    na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
    pred_label_id = predict(na_window)

    print ("Window Label: " + str(window_label_id) + "  Predicted Label: " + str(pred_label_id))
        
    total_preds += 1
        
    if (pred_label_id == window_label_id) or \
       (window_label_id>6 and pred_label_id==0):
        
        correct_preds += 1
        
    index = index + window_move_size
            
print ("\nContinuous Accuracy: " + str(correct_preds/total_preds))

In [33]:
run_lengths = []
current_label = 0
run_length = 1

for i in range(1 , len(df_OVA)):
    
    label = df_OVA.loc[i , 'label_id']
    
    if current_label == label:
        run_length += 1
        continue
    else:
        if current_label != 0:
            run_lengths.append(run_length)
        run_length = 1
        current_label = label
        continue

In [34]:
import matplotlib.pyplot as plt
run_lengths = np.array(run_lengths)
run_lengths_sorted = np.sort(run_lengths)
p = 1. * np.arange(len(run_lengths)) / (len(run_lengths) - 1)

plt.figure()
plt.plot(run_lengths_sorted, p)
plt.title('cumulative distribution function (CDF) of all run lengths')
plt.ylabel('p')
plt.xlabel('run length')
plt.show()

In [35]:
thr = 0.03
for i in range(p.shape[0]):
    if p[i] >= thr:
        break

idx = i-1
u = run_lengths_sorted[idx]
u

121

In [36]:
'''
thr = 0.03
for i in range(p.shape[0]):
    if p[i] >= thr:
        break

if p[i] == thr:
    u = run_lengths_sorted[i]
else:
    m = (p[i] - p[i-1]) / (run_lengths_sorted[i] - run_lengths_sorted[i-1])
    u = (thr - p[i-1] + m*run_lengths_sorted[i-1]) / m
    
u
'''

'\nthr = 0.03\nfor i in range(p.shape[0]):\n    if p[i] >= thr:\n        break\n\nif p[i] == thr:\n    u = run_lengths_sorted[i]\nelse:\n    m = (p[i] - p[i-1]) / (run_lengths_sorted[i] - run_lengths_sorted[i-1])\n    u = (thr - p[i-1] + m*run_lengths_sorted[i-1]) / m\n    \nu\n'

In [37]:
# =================
#df_continuous2 = df_continuous.loc[df_continuous['exp_id'] == 47]
df_continuous2 = df_continuous
# =================

models = []
models.append(model_1)
models.append(model_2)
models.append(model_3)
models.append(model_4)
models.append(model_5)
models.append(model_6)

index = 0
continu = True
correct_preds = 0
total_preds = 0
l = []
w = []
while (continu):
        
    if index >= len(df_continuous2):
        break
        
    df_window = df_continuous2.loc[index:index+window_size-1]
    if len(df_window) < window_size:
        break
        
    mode_win_labels = stats.mode(df_window['label_id'].values)
    window_label_id = mode_win_labels[0][0]
    
    if window_label_id == 0:
        print ("Window Label: 0")
        index = index + window_move_size
        continue
        
    na_window = np.array((df_window.loc[:,'acc0':'gyro2']))
    pred_label_id = predict(na_window)
    
    l.append(pred_label_id)
    w.append(window_label_id)
    
    i = len(l)
    if i > 2:
        
        pred_seg = [ l[i-3] , l[i-2] , l[i-1] ]
        mode = stats.mode(pred_seg)
        if mode[1][0] > 1:
            smoothed_pred_label_id = mode[0][0]
        else:
            smoothed_pred_label_id = l[i-2]
            
        print ("Window Label: " + str(w[i-2]) + "  Predicted Label: " + str(smoothed_pred_label_id))
      
        total_preds += 1
        
        if (smoothed_pred_label_id == w[i-2]) or \
           (w[i-2]>6 and smoothed_pred_label_id==0) or \
           (w[i-2]==7 and (smoothed_pred_label_id==5 or smoothed_pred_label_id==4)) or \
           (w[i-2]==8 and (smoothed_pred_label_id==4 or smoothed_pred_label_id==5)) or \
           (w[i-2]==9 and (smoothed_pred_label_id==4 or smoothed_pred_label_id==6)) or \
           (w[i-2]==10 and (smoothed_pred_label_id==6 or smoothed_pred_label_id==4)) or \
           (w[i-2]==11 and (smoothed_pred_label_id==5 or smoothed_pred_label_id==6)) or \
           (w[i-2]==12 and (smoothed_pred_label_id==6 or smoothed_pred_label_id==5)):
        
            correct_preds += 1
        
    index = index + window_move_size
            
print ("\nContinuous Accuracy: " + str(correct_preds/total_preds))

Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 7  Predicted Label: 4
Window Label: 7  Predicted Label: 1
Window Label: 7  Predicted Label: 1
Window Label: 7  Predicted Label: 1
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Wind

Window Label: 3  Predicted Label: 4
Window Label: 0
Window Label: 0
Window Label: 3  Predicted Label: 3
Window Label: 2  Predicted Label: 0
Window Label: 2  Predicted Label: 0
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 3
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 2  Predicted Label: 3
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label

Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 2  Predicted Label: 0
Window Label: 3  Predicted Label: 0
Window Label: 3  Predicted Label: 4
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 0
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 4
Window Label: 3  Predicted Label: 4
Window Label: 0
Window Label: 0
Window Label: 3  Predicted Label: 4
Window Label: 2  Predicted Label: 4
Window Label: 2  Predicted Label: 0
Window Label: 2  Predicted Label: 0
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Pre

Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 1  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 3  Pre

Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 12  Predicted Label: 6
Window Label: 12  Predicted Label: 6
Window Label: 12  Predicted Label: 6
Window Label: 12  Predicted Label: 4
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 12  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Window Label: 1  Predicted Label: 1
Win

...

Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 11  Predicted Label: 4
Window Label: 11  Predicted Label: 4
Window Label: 11  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicted Label: 6
Window Label: 6  Predicte

Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 8  Predicted Label: 1
Window Label: 8  Predicted Label: 1
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted L

Window Label: 2  Predicted Label: 0
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 1
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 5
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 5  Predicted Label: 4
Window Label: 7  Predicted Label: 4
Window Label: 7  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted Label: 4
Window Label: 4  Predicted L

Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 2  Predicted Label: 2
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 0
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 3  Predicted Label: 1
Window Label: 2  Predicted Label: 1
Window Label: 2  Predict

Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 3  Predicted Label: 1
Window Label: 2  Predicted Label: 1
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 2  Predicted Label: 2
Window Label: 0
Window Label: 0
Window Label: 0
Window Label: 2  Predicted Label: 2
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 1
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 3
Window Label: 3  Predicted Label: 1
Window Label: 3  Predict